# Avaliação de aprovação de cartão de crédito

O objetivo do projeto é criar um modelo de aprendizado de máquina capaz de prever se um pedido de cartão de crédito deve ser aprovado ou não com base em várias características do solicitante. 

As características usadas para prever a aprovação do cartão de crédito incluem informações como renda e origem da renda, gênero, nível de educação, estado civil, quantidade de filhos, tipo de moradia e se possui imóvel e/ou veículo próprio, entre outras.



# 1. Importando bibliotecas e lendo fontes de dados

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [ ]:
application = pd.read_csv("datasets/application_record.csv")
records = pd.read_csv("datasets/credit_record.csv")

# 2. Análise exploratória de dados

In [ ]:
application.info()

In [ ]:
records.info()

In [ ]:
application.head(10)

# CODE_GENDER >> Gênero
# FLAG_OWN_CAR >> Possui veículo próprio
# FLAG_OWN_REALTY >> Possui imóvel próprio
# CNT_CHILDREN >> Quantide de filhos
# AMT_INCOME_TOTAL >> Renda total
# NAME_INCOME_TYPE >> Origem da renda
# NAME_EDUCATION_TYPE >> Nível de educação
# NAME_FAMILY_STATUS >> Estado civil
# NAME_HOUSING_TYPE 
# DAYS_BIRTH >> Contagem regressiva de dias até a data de nascimento (-1 corresponde a ontem)
# DAYS_EMPLOYED >> Contagem regressiva de dias até a data de nascimento. Se positivo, significa que a pessoa está desempregada pelos dias correspondentes
# FLAG_MOBIL >> Possui telefone celular
# FLAG_WORK_PHONE >> Possui telefone celular corporativo
# FLAG_PHONE >> Possui telefone residencial
# FLAG_EMAIL >> Possui e-mail
# OCCUPATION_TYPE >> Profissão

In [ ]:
records.head()

# Cada ID possui uma entrada para cada mês desde sua aplicação, contendo na coluna 'STATUS' uma flag que corresponde a um determinado intervalo de dias
# que se passaram desde a contratação do crédito sem que houvesse pagamento

# MONTHS_BALANCE >> Contagem regressiva de meses desde a aplicação (-1 corresponde ao mês passado)
# STATUS >>
    # 0: 1-29 dias
    # 1: 30-59 dias
    # 2: 60-89 dias
    # 3: 90-119 dias
    # 4: 120-149 diasa
    # 5: Mais de 150 
    # C: Pago naquele mês
    # X: Sem debitos naquele mês

# 3. Transformação dos dados

In [ ]:
application['ID'].nunique() # Total de IDs únicos em "Applications" difere do total de linhas, ou seja, há dados duplicados

In [ ]:
application = application.drop_duplicates('ID', keep='last') # Eliminando dados duplicados, mantendo sempre o dado que aparece por último

In [ ]:
sns.heatmap(application.isnull()) # Verificando colunas com ocorrências de dados nulos em "Applications"

In [ ]:
sns.heatmap(records.isnull()) # Verificando colunas com ocorrências de dados nulos em "Records"

In [ ]:
application.drop(['OCCUPATION_TYPE'], axis=1, inplace=True) # Eliminando coluna 'OCCUPATION_TYPE' que possui entradas com valores nulos

In [ ]:
le = LabelEncoder()
for x in application:
    if application[x].dtypes=='object':
        application[x] = le.fit_transform(application[x])
        
# Transformando dados não numéricos em numéricos

In [ ]:
fig, ax= plt.subplots(nrows= 3, ncols = 3, figsize= (14,6))

sns.scatterplot(x='ID', y='CNT_CHILDREN', data=application, ax=ax[0][0], color= 'red')
sns.scatterplot(x='ID', y='AMT_INCOME_TOTAL', data=application, ax=ax[0][1], color='red')
sns.scatterplot(x='ID', y='DAYS_BIRTH', data=application, ax=ax[0][2])
sns.scatterplot(x='ID', y='DAYS_EMPLOYED', data=application, ax=ax[1][0])
sns.scatterplot(x='ID', y='FLAG_MOBIL', data=application, ax=ax[1][1])
sns.scatterplot(x='ID', y='FLAG_WORK_PHONE', data=application, ax=ax[1][2])
sns.scatterplot(x='ID', y='FLAG_PHONE', data=application, ax=ax[2][0])
sns.scatterplot(x='ID', y='FLAG_EMAIL', data=application, ax=ax[2][1])
sns.scatterplot(x='ID', y='CNT_FAM_MEMBERS', data=application, ax=ax[2][2], color= 'red')

# Verificando outliers (em vermelho as colunas com ocorrências de outliers)

In [ ]:
out_max = application['CNT_CHILDREN'].quantile(0.999)
out_min = application['CNT_CHILDREN'].quantile(0.001)
application = application[(application['CNT_CHILDREN']>out_min) & (application['CNT_CHILDREN']<out_max)]

# Eliminando outliers em "CNT_CHILDREN"

In [ ]:
out_max = application['AMT_INCOME_TOTAL'].quantile(0.999)
out_min = application['AMT_INCOME_TOTAL'].quantile(0.001)
application = application[(application['AMT_INCOME_TOTAL']>out_min) & (application['AMT_INCOME_TOTAL']<out_max)]

# Eliminando outliers em "AMT_INCOME_TOTAL"

In [ ]:
out_max = application['CNT_FAM_MEMBERS'].quantile(0.999)
out_min = application['CNT_FAM_MEMBERS'].quantile(0.001)
application = application[(application['CNT_FAM_MEMBERS']>out_min) & (application['CNT_CHILDREN']<out_max)]

# Eliminando outliers em "CNT_FAM_MEMBERS"

In [ ]:
fig, ax= plt.subplots(nrows= 3, ncols = 3, figsize= (14,6))

sns.scatterplot(x='ID', y='CNT_CHILDREN', data=application, ax=ax[0][0], color= 'green')
sns.scatterplot(x='ID', y='AMT_INCOME_TOTAL', data=application, ax=ax[0][1], color='green')
sns.scatterplot(x='ID', y='DAYS_BIRTH', data=application, ax=ax[0][2])
sns.scatterplot(x='ID', y='DAYS_EMPLOYED', data=application, ax=ax[1][0])
sns.scatterplot(x='ID', y='FLAG_MOBIL', data=application, ax=ax[1][1])
sns.scatterplot(x='ID', y='FLAG_WORK_PHONE', data=application, ax=ax[1][2])
sns.scatterplot(x='ID', y='FLAG_PHONE', data=application, ax=ax[2][0])
sns.scatterplot(x='ID', y='FLAG_EMAIL', data=application, ax=ax[2][1])
sns.scatterplot(x='ID', y='CNT_FAM_MEMBERS', data=application, ax=ax[2][2], color= 'green')

# Dados após eliminação dos outliers

In [ ]:
records['STATUS'].replace({'C': 0, 'X' : 0}, inplace=True)
records['STATUS'] = records['STATUS'].astype('int')
records['STATUS'] = records['STATUS'].apply(lambda x:1 if x >= 2 else 0)
records = records.groupby('ID').agg(max).reset_index()

# Substituindo valores "C" e "X" da coluna "STATUS" por 0, pois possuem a mesma equivalência (estão em dia)
# Aplicando regra de corte para valores com "STATUS" maior que 2, ou seja, mais de 60 dias em atraso
# Mantendo apenas um registro por ID, constando um status que apresenta o máximo de dias que este ID esteve inadimplente

In [ ]:
application.head(10)

In [ ]:
records.head(10)

In [ ]:
len(records) # Total de linhas em "Records"

In [ ]:
len(application) # Total de linhas em "Application"

In [ ]:
df = application.join(records.set_index('ID'), on='ID', how='inner')
df.drop(['MONTHS_BALANCE'], axis=1, inplace=True)

# Juntando "Application" com "Records" em um novo dataframe

In [ ]:
df.head(10)

In [ ]:
len(df)  # Total de linhas do novo dataframe

In [ ]:
df['STATUS'].value_counts(normalize=True) # Verificando se há superamostragem

# 4. Desenvolvendo o modelo

In [ ]:
x = df.iloc[:,1:-1] # Colunas exceto as de classificação
y = df.iloc[:,-1] # Colunas de classificação

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3) # Dividindo os dados entre teste e treino

In [ ]:
mms = MinMaxScaler()
x_scaled = pd.DataFrame(mms.fit_transform(x_train), columns=x_train.columns)
x_test_scaled = pd.DataFrame(mms.transform(x_test), columns=x_test.columns)

# Escalando dados para um resultado mais acurado

In [ ]:
oversample = SMOTE()
x_balanced, y_balanced = oversample.fit_resample(x_scaled, y_train)
x_test_balanced, y_test_balanced = oversample.fit_resample(x_test_scaled, y_test)

# Lidando com o problema de superamostragem

In [ ]:
classifiers = {
    "LogisticRegression" : LogisticRegression(),
    "KNeighbors" : KNeighborsClassifier(),
    "SVC" : SVC(),
    "DecisionTree" : DecisionTreeClassifier(),
    "RandomForest" : RandomForestClassifier(),
    "XGBoost" : XGBClassifier()
}

# Definindo modelos a serem testados

In [ ]:
train_scores = []
test_scores = []

for key, classifier in classifiers.items():
    classifier.fit(x_balanced, y_balanced)
    train_score = classifier.score(x_balanced, y_balanced)
    train_scores.append(train_score)
    test_score = classifier.score(x_test_balanced, y_test_balanced)
    test_scores.append(test_score)

print(train_scores)
print(test_scores)

# Executando os modelos e verificando acurácia

In [ ]:
xgb = XGBClassifier()
model = xgb.fit(x_balanced, y_balanced)
prediction = xgb.predict(x_test_balanced)

# Escolhendo o melhor modelo = XGBoost

In [ ]:
print(classification_report(y_test_balanced, prediction))

# Resultado do modelo

# Conclusão

Após testes, concluiu-se que o modelo XGBoost obteve melhor acurácia.

Entendeu-se e definiu-se que o modelo basearia-se nas aplicações que ficaram, pelo menos uma vez, inadimplentes com mais de 60 dias de atraso para reprovar a análise de crédito.